# OCR Integration for Scanned Documents

Optical Character Recognition (OCR) allows you to extract text from scanned documents where the text isn't embedded in the PDF. This tutorial demonstrates how to work with scanned documents.

In [1]:
#%pip install "natural-pdf[all]"

In [2]:
from natural_pdf import PDF

# Load a PDF
pdf = PDF("https://github.com/jsoma/natural-pdf/raw/refs/heads/main/pdfs/needs-ocr.pdf")
page = pdf.pages[0]

# Try extracting text without OCR
text_without_ocr = page.extract_text()
f"Without OCR: {len(text_without_ocr)} characters extracted"

'Without OCR: 0 characters extracted'

## Enabling OCR

In [3]:
# Enable OCR for text extraction
page.use_ocr = True

# Extract text with OCR enabled
text_with_ocr = page.extract_text()

# Preview the extracted text
text_with_ocr[:200] + "..." if len(text_with_ocr) > 200 else text_with_ocr

''

## Finding Text Elements with OCR

In [4]:
# Convert text-as-image to text elements
page.apply_ocr()

# Select all text pieces on the page
text_elements = page.find_all('text')
f"Found {len(text_elements)} text elements"

# Visualize the elements
text_elements.highlight()

2025-04-12T15:14:07.218844Z [warning  ] Using CPU. Note: This module is much faster with a GPU. lineno=71 module=easyocr.easyocr


[2025-04-12 17:14:07,218] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.


<ElementCollection[Mixed](count=0)>

## OCR Configuration Options

In [5]:
# Set OCR configuration for better results
page.ocr_config = {
    'language': 'eng',  # English
    'dpi': 300,         # Higher resolution
}

# Extract text with the improved configuration
improved_text = page.extract_text()

# Preview the text
improved_text[:200] + "..." if len(improved_text) > 200 else improved_text

''

## Working with Multi-language Documents

In [6]:
# Configure for multiple languages
page.ocr_config = {
    'language': 'eng+fra+deu',  # English, French, German
    'dpi': 300
}

# Extract text with multi-language support
multilang_text = page.extract_text()
multilang_text[:200]

''

## Extracting Tables from Scanned Documents

In [7]:
# Enable OCR and analyze the document layout
page.use_ocr = True
page.analyze_layout()

# Find table regions
table_regions = page.find_all('region[type=table]')

# Visualize any detected tables
table_regions.highlight()

# Extract the first table if found
if table_regions:
    table_data = table_regions[0].extract_table()
    table_data
else:
    "No tables found in the document"

image 1/1 /var/folders/25/h3prywj14qb0mlkl2s8bxq5m0000gn/T/tmpkipu47zn/temp_layout_image.png: 1024x800 2 titles, 2 plain texts, 3 abandons, 1 table, 2141.2ms


Speed: 8.2ms preprocess, 2141.2ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 800)


## Finding Form Fields in Scanned Documents

In [8]:
# Look for potential form labels (containing a colon)
labels = page.find_all('text:contains(":")') 

# Visualize the labels
labels.highlight()

# Extract form data by looking to the right of each label
form_data = {}
for label in labels:
    # Clean the label text
    field_name = label.text.strip().rstrip(':')
    
    # Find the value to the right
    value_element = label.right(width=200)
    value = value_element.extract_text().strip()
    
    # Add to our dictionary
    form_data[field_name] = value

# Display the extracted data
form_data

{}

## Combining OCR with Layout Analysis

In [9]:
# Apply OCR and analyze layout
page.use_ocr = True
page.analyze_layout()

# Find document structure elements
headings = page.find_all('region[type=heading]')
paragraphs = page.find_all('region[type=paragraph]')

# Visualize the structure
headings.highlight(color="red", label="Headings")
paragraphs.highlight(color="blue", label="Paragraphs")

# Create a simple document outline
document_outline = []
for heading in headings:
    heading_text = heading.extract_text()
    document_outline.append(heading_text)

document_outline

image 1/1 /var/folders/25/h3prywj14qb0mlkl2s8bxq5m0000gn/T/tmpezhw3fc2/temp_layout_image.png: 1024x800 2 titles, 2 plain texts, 3 abandons, 1 table, 1728.8ms


Speed: 4.7ms preprocess, 1728.8ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 800)


[]

## Working with Multiple Pages

In [10]:
# Process all pages in the document
all_text = []

for i, page in enumerate(pdf.pages):
    # Enable OCR for each page
    page.use_ocr = True
    
    # Extract text
    page_text = page.extract_text()
    
    # Add to our collection with page number
    all_text.append(f"Page {i+1}: {page_text[:100]}...")

# Show the first few pages
all_text

['Page 1: ...']

## Saving PDFs with Searchable Text

After applying OCR to the pages of a PDF (e.g., using `pdf.apply_ocr_to_pages()`), you can save a new version of the PDF where the recognized text is embedded as an invisible layer. This makes the text searchable and copyable in standard PDF viewers.

Use the `save_searchable()` method on the `PDF` object:

In [11]:
from natural_pdf import PDF

input_pdf_path = "https://github.com/jsoma/natural-pdf/raw/refs/heads/main/pdfs/needs-ocr.pdf"
output_pdf_path = "needs-ocr-searchable.pdf"

# Load the PDF
pdf = PDF(input_pdf_path)

# Apply OCR to all pages (this generates the text elements needed for saving)
# Replace 'easyocr' with 'paddle' or 'surya' if desired
print("Applying OCR...")
pdf.apply_ocr_to_pages(engine='easyocr') 

# Save the searchable version
print(f"Saving searchable PDF to {output_pdf_path}...")
pdf.save_searchable(output_pdf_path)

print("Done. Check needs-ocr-searchable.pdf")

# Optional: Close the original PDF
pdf.close()

2025-04-12T15:14:21.910988Z [warning  ] Using CPU. Note: This module is much faster with a GPU. lineno=71 module=easyocr.easyocr


[2025-04-12 17:14:21,910] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.


Applying OCR...


Saving searchable PDF to needs-ocr-searchable.pdf...


2025-04-12T15:14:32.177661Z [warning  ] Page 1 has no OCR elements (text[source=ocr]) to generate hOCR from. lineno=86 module=natural_pdf.exporters.searchable_pdf pageno=None


[2025-04-12 17:14:32,177] [ WARNING] searchable_pdf.py:86 - Page 1 has no OCR elements (text[source=ocr]) to generate hOCR from.


Done. Check needs-ocr-searchable.pdf


This creates `needs-ocr-searchable.pdf`, which looks identical to the original but now has a text layer corresponding to the OCR results. You can adjust the rendering resolution used during saving with the `dpi` parameter (default is 300).

OCR integration enables you to work with scanned documents, historical archives, and image-based PDFs that don't have embedded text. By combining OCR with natural-pdf's layout analysis capabilities, you can turn any document into structured, searchable data. 